## 1:Import Data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
from pandas_profiling import ProfileReport  

import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.iolib.summary2 import summary_col
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV,Lasso
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split,RepeatedKFold,GridSearchCV
from sklearn.neighbors import NearestNeighbors
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression as lr
from sklearn.compose import ColumnTransformer
from sklearn import set_config

import statsmodels.formula.api as smf
import math
from functions import *
set_config(display="diagram")

#import warnings
#warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
# We define any blank value as NA 
missing_values_char = ['']
df = pd.read_excel('数据/dataset_version5_Final.xlsx',na_values=missing_values_char)
dfclean = df.copy()

## 2: Data Clean and pre-process

###### 2-1 delivery_type_1

In [ ]:
# Strip the column for delivery_type_1
dfclean.delivery_type_1 = dfclean.delivery_type_1.str.strip()

In [ ]:
# Replace value with ENG
dfclean['delivery_type_1'] = dfclean['delivery_type_1'].replace(['美团快送'],'MeituanFlash')
dfclean['delivery_type_1'] = dfclean['delivery_type_1'].replace(['美团跑腿'],'MeituanRun')
dfclean['delivery_type_1'] = dfclean['delivery_type_1'].replace(['商家'],'Merchant')

In [ ]:
# Drop delivery_type NA value
dfclean = dfclean.dropna(subset=['delivery_type_1'])

###### 2-2 category1
```
Did not slice the category1 feature into subgroups. Try to keep the integrity of the data set 
as much as possible
```

In [ ]:
# Strip the column for category1
dfclean.category1 = dfclean.category1.str.strip()

In [ ]:
# Replace value with ENG
dfclean['category1'] = dfclean['category1'].replace(['超市便利'],'MarketConvenience')
dfclean['category1'] = dfclean['category1'].replace(['浪漫鲜花'],'RomanticFlowers')
dfclean['category1'] = dfclean['category1'].replace(['美食'],'Foods')
dfclean['category1'] = dfclean['category1'].replace(['生鲜果蔬'],'FruitVegetables')
dfclean['category1'] = dfclean['category1'].replace(['送药上门'],'DrugsDeliveried')
dfclean['category1'] = dfclean['category1'].replace(['甜点饮品'],'DessertDrink')
dfclean['category1'] = dfclean['category1'].replace(['甜蜜蛋糕'],'SweetCake')
dfclean['category1'] = dfclean['category1'].replace(['未知'],'Unkown')

In [ ]:
# Drop category1 "未知" value
dfclean = dfclean[dfclean.category1!='Unkown']

###### 2-3 in_time_delivery_percent
```
13% of in_time_delivery_percent values are zero. Yet, I investigate the features distributions under in_time_delivery_percent==0.
Most of the distributions actually make sense. in_time_delivery_percent == 0 just means it takes a long time to deliver. So these many zero valeus actually means something, decide not to drop them
```

In [ ]:
# There are 800s NA value in_time_delivery_percent feature, since this feature is curical in the upcoming data minning
# Drop all NA value in this feature
dfclean = dfclean.dropna(subset=['in_time_delivery_percent'])

###### 2-4 delivery_tip

In [ ]:
# Drop the delivery tip feature, since we already have delivery_type_1 
dfclean.drop(columns=['delivery_tip'],inplace=True)

###### 2-5 score

In [ ]:
# Drop score equal to zero 
# score is the output variable candidate, 0 will affact the model significantly
dfclean = dfclean[dfclean.score>0]

###### 2-6 month_sales_clean
```
Not sure how to deal with the extrem value, may log the whole feature
```

In [ ]:
# Drop month_sales_clean equal to zero 
# month_sales_clean is the output variable candidate, 0 will affact the model significantly
dfclean = dfclean[dfclean.month_sales_clean>0]

###### 2-7 min_price_rmb
```
15% of min_price_rmb values are zero. Yet, I investigate the features distributions under min_price_rmb==0.
Most of the distributions actually make sense. min_price_rmb == 0 just means no barrier to make an order. So these many zero valeus actually means something, decide not to drop them
```

###### 2-8 comment_num

In [ ]:
# only 0.1% of the values in comment_number are zero, we will drop them
dfclean = dfclean[dfclean.comment_number>0]

###### 2-9 delivery_time_clean

In [ ]:
# Drop the delivery_time_clean <= 300 mins
dfclean = dfclean[dfclean.delivery_time_clean<=300]

##### 2-10delivery_type_bin

In [ ]:
# Create binary variable for delivery_type
# Merchant = 1 else 0
dfclean[['MeituanFlash','MeituanRun','Merchant']] = pd.get_dummies(dfclean['delivery_type_1'])
dfclean['delivery_type_bin'] = dfclean['Merchant']

##### 2-11 month_sales_clean_log

In [ ]:
# Perform the log transformation for the month_sales_clean
dfclean['month_sales_clean_log'] = np.log(dfclean.month_sales_clean)

In [ ]:
# Visulize and compared the log and non-log version
fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(4, 3), dpi=100)
plt.tight_layout()

axs[0].hist(dfclean.month_sales_clean, bins=100)
axs[1].hist(dfclean.month_sales_clean_log, bins=100)

##### 2-12 Create subset by limiting category (foods & dessertdrink &sweetcake）

In [ ]:
dffood=dfclean.drop(dfclean[dfclean['category1']=='MarketConvenience'].index)
dffood=dffood.drop(dffood[dffood['category1']=='FruitVegetables'].index)
dffood=dffood.drop(dffood[dffood['category1']=='RomanticFlowers'].index)
dffood=dffood.drop(dffood[dffood['category1']=='DrugsDeliveried'].index)
dffood['category1'].value_counts()

##### 2-13 category1 dummy (one hot encode)

In [ ]:
dfclean[['DessertDrink','DrugsDeliveried','Foods','FruitVegetables','MarketConvenience','RomanticFlowers','SweetCake']] = pd.get_dummies(dfclean['category1'])

##### 2-14 Exclude Education & Beauty 

In [ ]:
# education and beauty are sparse features which have tons of zero value. They won't add too much information in the model
# so we will drop these two features
dfclean.drop(columns=['Education'],inplace=True)
dfclean.drop(columns=['Beauty'],inplace=True)

##### 2-14 Export Data-Set

In [ ]:
#dfclean.to_excel('datasetv7_allcategory.xlsx',index = False)

In [ ]:
#dffood.to_excel('datasetv8_food_drink_cake.xlsx',index = False)

## 3 EDA 
```
Use these codes to do a sanitiy check for the cleanning
```

In [ ]:
profile = ProfileReport(dffood, title="Pandas Profiling Report")
profile

## 4: Visualization  

```
Refer to the correlation plot in the Auto EDA result, visualize features pairs of following
in_time_delivery_percent -- delivery_time_clean 
delivery_time_clean      --- min_price_rmb
comment_number           --- month_sales_clean_log 
day_time                 --- mid_night_time
catergory1               --- avg_price_rmb
delivery_type_1          --- avg_price_rmb
```

In [ ]:
sns.set(rc={"figure.dpi":80, 'savefig.dpi':80})

###### 4-1 in_time_delivery_percent -- delivery_time_clean 

In [ ]:
sns.set_style('dark')
axes = sns.jointplot(
    data=dfclean,
    x="in_time_delivery_percent",
    y="delivery_time_clean",
    hue="delivery_type_bin",
    hue_order=[1, 0],
    color="navy",
    palette="bright",
    #kind='hist'
)

##### 4-2 delivery_time_clean      --- min_price_rmb

In [ ]:
sns.set_style('dark')
axes = sns.jointplot(
    data=dfclean,
    x="delivery_time_clean",
    y="min_price_rmb",
    hue="delivery_type_bin",
    hue_order=[1, 0],
    color="navy",
    palette="bright",
    #kind='hist'
)

##### 4-3 comment_number           --- month_sales_clean_log 

In [ ]:
sns.set_style('dark')
axes = sns.jointplot(
    data=dfclean,
    x="comment_number",
    y="month_sales_clean_log",
    hue="delivery_type_bin",
    hue_order=[1, 0],
    color="navy",
    palette="bright",
    #kind='hist'
)

##### 4-4 catergory1               --- avg_price_rmb

In [ ]:
sns.set(rc={"figure.dpi":80, 'savefig.dpi':80})
sns.set(rc={'figure.figsize':(12,4)})
sns.set_style('dark')
sns.boxplot(data=dfclean, x='category1',y='avg_price_rmb',palette='bright')

##### 4-5 delivery_type_1          --- avg_price_rmb

In [ ]:
sns.set(rc={"figure.dpi":60, 'savefig.dpi':60})
sns.set(rc={'figure.figsize':(12,7)})
sns.set_style('dark')
sns.boxplot(data=dfclean, x='delivery_type_1',y='avg_price_rmb',palette='bright')

## 5: Model

### 5-1 Correlation Heatmap

In [ ]:
plt.figure(figsize=(11,8))
corrMatrix = dffood.corr()
sns.heatmap(corrMatrix, annot=True)
plt.title('Correlation Matrix')
plt.show()

In [ ]:
dffood.month_sales_clean.hist() # right skewed

In [ ]:
dffood.month_sales_clean_log.hist() # looks normal

### 5-2 Linear Regression
The distribution of Y has a normal distribution with mean  μ and constant variance σ^2.  
Link function - the identity link, η=g(E(Y))=E(Y), is used; this is the simplest link function.

#### 5-2-1 level-level regression

In [ ]:
results1 = ols('month_sales_clean ~ score+comment_number+avg_price_rmb+delivery_time_clean+delivery_type_1+in_time_delivery_percent+min_price_rmb+shipping_fee_clean+Day_Time+Night_Time+Mid_Night_Time+recommend_Bin', data=dffood).fit()
print(results1.summary())

In [ ]:
MSE1=(np.square(results1.resid)).mean()

We can write our simple linear regression model as

$$
{MonthlySales}_i = \beta_0 + \beta_1 {DeliveryType[T.MeituanRun]}_i + \beta_2 {DeliveryType_1[T.Merchant]}_i+ \beta_3 {Score}_i+ \beta_4 {CommentNumber}_i+\beta_5 {AvgPrice}_i+\beta_6 {DeliveryTime}_i+\beta_7 {InTimeDeliveryPercent}_i+\beta_8 {MinPrice}_i+\beta_9 {ShippingFee}_i+\beta_{10} {DayTime}_i+\beta_{11} {NightTime}_i+\beta_{12} {MidNightTime}_i+\beta_{13} {Recommendation}+u_i
$$



#### 5-2-2 Log-linear regression

In [ ]:
results2 = ols('month_sales_clean_log ~ score+comment_number+avg_price_rmb+delivery_time_clean+delivery_type_1+in_time_delivery_percent+min_price_rmb+shipping_fee_clean+Day_Time+Night_Time+Mid_Night_Time+recommend_Bin', data=dffood).fit()
#print(results2.summary())

results3 = ols('month_sales_clean_log ~ score+comment_number+avg_price_rmb+delivery_time_clean+delivery_type_1+in_time_delivery_percent+min_price_rmb+shipping_fee_clean+Day_Time+Mid_Night_Time+recommend_Bin', data=dffood).fit()
#print(results3.summary())

In [ ]:
from statsmodels.iolib.summary2 import summary_col
info_dict={'R-squared' : lambda x: f"{x.rsquared:.2f}", 
           'No. observations' : lambda x: f"{int(x.nobs):d}"}


results_table = summary_col(results=[results2,results3],
                            float_format='%0.3f',
                            stars = True,
                            model_names=['Model 2',
                                         'Model 3'],
                            info_dict=info_dict,
                            regressor_order=['Intercept',
                                             'delivery_type_1[T.MeituanRun]',
                                             'delivery_type_1[T.Merchant]',
                                             'score',
                                             'comment_number',
                                             'avg_price_rmb',
                                             'delivery_time_clean'])
                            

results_table.add_title('Table 4-2 - OLS Regressions Summaries for Model 1-5')

print(results_table)

In [ ]:
MSE2=(np.square(results2.resid)).mean()  
MSE3=(np.square(results3.resid)).mean()

Analysis Assumptions:  
1.Gauss markov assumption hold  
2.Statistically significant  
3.Hold all other indep variable constant

EXAMPLE: Hold all other indepdent variable constant, every one-unit increase in score, y increases by about 43%

### 5-3 GLM Model  :   
Log Link: Log(Y) – Models the logarithm of mean Y. 

In [ ]:
formula = 'month_sales_clean ~ score+comment_number+avg_price_rmb+delivery_time_clean+delivery_type_1+in_time_delivery_percent+min_price_rmb+shipping_fee_clean+Day_Time+Night_Time+Mid_Night_Time+recommend_Bin'

#### 5-3-1 Poisson GLM

In [ ]:
model4 = smf.glm(formula = formula, data=dffood, family=sm.families.Poisson(sm.families.links.log()))
results4 = model4.fit()
print(results4.summary())

WE have overdispersion (i.e. residual deviance is much larger than degrees of freedom), we want to use quasipoisson() instead of poisson().

In [ ]:
MSE4=np.square(results4.resid_response).mean()

#### 5-3-2 Gaussian GLM

In [ ]:
model5 = smf.glm(formula = formula, data=dffood, family=sm.families.Gaussian(sm.families.links.log()))
results5 = model5.fit()
print(results5.summary())

In [ ]:
MSE5=np.square(results5.resid_response).mean()

### 5-4 Model Comparison

#### 5-4-1 MSE

Mean square error (MSE) is the average of the square of the errors. The larger the number the larger the error. Error in this case means the difference between the observed values y1, y2, y3, … and the predicted ones pred(y1), pred(y2), pred(y3), … We square each difference (pred(yn) – yn)) ^ 2 so that negative and positive values do not cancel each other out.  
Simply put, the lower the value the better and 0 means the model is perfect. 

In [ ]:
print(MSE1) #level-level
print(MSE2) #log-linear with Night_Time
print(MSE3) #log-linear without Night_Time
print(MSE4) #glm pois
print(MSE5) #glm gaussian

这里result 和r 不一样 idk why

We find that log-linear with Night_Time model and log-linear without Night_Time model is better than other models

#### 5-4-2 AIC and R^2

The Akaike information criterion (AIC) is a mathematical method for evaluating how well a model fits the data it was generated from. In statistics, AIC is used to compare different possible models and determine which one is the best fit for the data.   
AIC is calculated from:  
the number of independent variables used to build the model   
the maximum likelihood estimate of the model (how well the model reproduces the data)  
  
The best-fit model according to AIC is the one that explains the greatest amount of variation using the fewest possible independent variables.

Lower AIC values indicate a better-fit model, and a model with a delta-AIC (the difference between the two AIC values being compared) of more than -2 is considered significantly better than the model it is being compared to.

In [ ]:
print(results2.aic)
print(results3.aic)

In [ ]:
print(results2.rsquared)
print(results3.rsquared)

### Scatter fit plot

In [ ]:
# scatter fit plot
Yhat= results2.fittedvalues
Y=dffood.month_sales_clean_log
plt.scatter(Yhat,Y)

Based on MSE result, R^2 and scatter plt, model 2 is the best.
$$
{log(MonthlySales)}_i = \beta_0 + \beta_1 {DeliveryType[T.MeituanRun]}_i + \beta_2 {DeliveryType_1[T.Merchant]}_i+ \beta_3 {Score}_i+ \beta_4 {CommentNumber}_i+\beta_5 {AvgPrice}_i+\beta_6 {DeliveryTime}_i+\beta_7 {InTimeDeliveryPercent}_i+\beta_8 {MinPrice}_i+\beta_9 {ShippingFee}_i+\beta_{10} {DayTime}_i+\beta_{11} {NightTime}_i+\beta_{12} {MidNightTime}_i+\beta_{13} {Recommendation}+u_i
$$



In [ ]:
print(results2.summary())

### 5-2 LASSO--Full Model(Baseline)(Include all features)

```
lasso regression’s advantage over least squares linear regression is rooted in the bias-variance trade-off. As α increases, the flexibility of the lasso regression fit decreases, leading to decreased variance but increased bias. This procedure is more restrictive in estimating the coefficients and - depending on your value of α may set a number of them to exactly zero. This means in the final model the response variable will only be related to a small subset of the predictors—namely, those with nonzero coeffcient estimates. Therefore, selecting a good value of α is critical.
```

In [ ]:
# Define X (independent variable)
# We will exclude the geo info data, such as province, city, district, lat & lng
# We will exclude the month_sales_clean & log. Since this is output variable
# For delivery_type, use Merchant as base
# For category type, use DrugsDelivered as base since it only very few data                       
X = dfclean.drop(['province',
                  'city',
                  'district',
                  'name',
                  'month_sales_clean',
                  'category1',
                  'delivery_type_1',
                  'lat',
                  'lng',
                  'trade_area',
                  'Merchant',
                  'delivery_type_bin',
                  'month_sales_clean_log',
                  'DrugsDeliveried'],axis=1)

In [ ]:
# Define Y (output variable)
# Use month_sales_clean_log as target variable
y = dfclean['month_sales_clean_log']

In [ ]:
# This is all the numerical features 
numeric_features = ['score',
                    'comment_number',
                    'avg_price_rmb',
                    'delivery_time_clean',
                    'in_time_delivery_percent',
                    'min_price_rmb',
                    'shipping_fee_clean',
                    'district population in (10 thousands)',
                    'population density (people/square kilometer)',
                    'Rent',
                    'Female',
                    'Male',
                    '18 and below',
                    '19-24',
                    '25-34',
                    '35-44',
                    '45-54',
                    '55 and above',
                    '50 yuan and below',
                    '50-100 yuan',
                    '100-300 yuan',
                    '300 yuan',
                    'Shopping',
                    'Catering Service',
                    'Car Service',
                    'Leisure',
                    'Other',
                    'Medicare']

In [ ]:
# Train - test split 80% - 20% 
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

# Normalize the X_train data in X 
scaler = StandardScaler()
X_train[numeric_features] = scaler.fit_transform(X_train[numeric_features])
X_test[numeric_features] = scaler.fit_transform(X_test[numeric_features])

# Define evaluation method
cv = RepeatedKFold(n_splits=10,n_repeats = 3)

# Define model
model = LassoCV(cv=cv,n_jobs=-1)

# Fit model
model.fit(X_train,y_train)

In [ ]:
# Extract optimal alpha
optimal_alpha = model.alpha_

# Fit the lasso with optimal alpha 
lasso_best = Lasso(alpha=optimal_alpha)
lasso_best.fit(X_train, y_train)

print(f'Optimal Alpha: {optimal_alpha}')
print('R squared training set', round(lasso_best.score(X_train, y_train)*100, 2))
print('R squared test set', round(lasso_best.score(X_test, y_test)*100, 2))

plt.semilogx(model.alphas_, model.mse_path_, ":")
plt.plot(
    model.alphas_ ,
    model.mse_path_.mean(axis=-1),
    "k",
    label="Average across the folds",
    linewidth=2,
)
plt.axvline(
    model.alpha_, linestyle="--", color="k", label="alpha: CV estimate"
)

plt.legend()
plt.xlabel("alphas")
plt.ylabel("Mean square error")
plt.title("Mean square error on each fold")
plt.axis("tight")

ymin, ymax = 1, 1.5
plt.ylim(ymin, ymax);

In [ ]:
# Extract coefficient from best lasso model
for i,j in zip(lasso_best.coef_,X_train.columns):
    print(round(i,2),"       ",j)

## 5-3 P-Score Matching
### Treatment = Merchant Vs NonMerchant 
### Subgroup= All Category1
### Y = month_sale_clean

In [ ]:
pd.pivot_table(dfclean,
              values = ['month_sales_clean','comment_number','avg_price_rmb','in_time_delivery_percent','delivery_time_clean'],
              index = 'delivery_type_bin',
               aggfunc=np.mean)

#### 5-3-1: P-Score

In [ ]:
# Define X & Y for P-score prediction

# Base on the LassoCV, we extract the feature that is non zero
# Remind that we will include all the dummy variable of category1 feature regardless the lasso result
Important_Feature = ['score',
                     'comment_number',
                     'avg_price_rmb',
                    'delivery_time_clean',
                    'in_time_delivery_percent',
                     'min_price_rmb',
                    'shipping_fee_clean',
                    'Day_Time',
                    'Night_Time',
                    'Mid_Night_Time',
                    'recommend_Bin',
                     'district population in (10 thousands)',
                     'population density (people/square kilometer)',
                     'Rent',
                     'Female',
                     '18 and below',
                     '19-24',
                     '25-34',
                     '35-44',
                     '45-54',
                     '55 and above',
                     '50-100 yuan',
                     '300 yuan',
                     'Shopping',
                     'Catering Service',
                     'Car Service',
                     'Leisure',
                     'Other',
                     'Medicare',
                    'MeituanFlash',
                    'MeituanRun',
                     'DessertDrink',
                     'DrugsDeliveried',
                    'Foods',
                     'FruitVegetables',
                    'MarketConvenience',
                     'RomanticFlowers',
                    'SweetCake',]

# For Y
# The treatment we use is delivery_type_bin
# 1 = Merchant 0 = Non-Merchant
# Estimatae the Treatment Effect between Merchant and Meituan(Run+Flash) 
T = dfclean.delivery_type_bin


# For X
# 1: Regarding the input variable, we use important feature generated from above lasso regression where coefficient is non-zero
# 2: Make sure it MUST NOT contain the treatment
#    Therefore we drop 'MeituanRun' and 'MeituanFlash', cause we are predicting the delivery type as our P-Score(Which is same as Merchant)
# 3: Since the Output of Matching analysis is the monthly sales data, we need to make sure that X MUST NOT contain 'month_sales_clean' and 'month_sales_clean_log' 
# 4: Remind that we will include all the dummy variable of category1 feature regardless the lasso result
Important_Feature.remove('MeituanRun')
Important_Feature.remove('MeituanFlash')
X = dfclean[Important_Feature]

In [ ]:
# Design pipline 
numeric_features= ['score',
                    'comment_number',
                    'avg_price_rmb',
                    'delivery_time_clean',
                    'in_time_delivery_percent',
                    'min_price_rmb',
                    'shipping_fee_clean',
                    'district population in (10 thousands)',
                    'population density (people/square kilometer)',
                    'Rent',
                    'Female',
                    '18 and below',
                    '19-24',
                    '25-34',
                    '35-44',
                    '45-54',
                    '55 and above',
                    '50-100 yuan',
                    '300 yuan',
                    'Shopping',
                    'Catering Service',
                    'Car Service',
                    'Leisure',
                    'Other',
                    'Medicare']


numeric_transformer  = Pipeline(steps=[('Scaler',StandardScaler())])
preprocessor         = ColumnTransformer(transformers=[('num',numeric_transformer,numeric_features)])
clf                  = Pipeline(steps=[('preprocessor',preprocessor),("classifier",lr())])
grid                 = {"classifier__C": np.arange(1,10,0.1).tolist()}
grid_search          = GridSearchCV(clf,grid,cv=10)
grid_search.fit(X,T)

In [ ]:
# Use the optimal parameter in logist
# Output the prediction base on the best parameter
BestPip = Pipeline(steps=[('preprocessor',preprocessor),
                          ("classifier",lr(C=grid_search.best_params_['classifier__C']))])
BestPip.fit(X, T)
predictions = BestPip.predict_proba(X)
predictions_binary = BestPip.predict(X)

# Eavluate the model
print('Accuracy: {:.4f}\n'.format(metrics.accuracy_score(T, predictions_binary)))
print('Precision: {:.4f}\n'.format(metrics.precision_score(T, predictions_binary)))
print('Confusion matrix:\n{}\n'.format(metrics.confusion_matrix(T, predictions_binary)))
print('F1 score is: {:.4f}'.format(metrics.f1_score(T, predictions_binary)))

In [ ]:
def logit(p):
    logit_value = math.log(p / (1-p))
    return logit_value
sns.set(rc={'figure.figsize':(16,10)}, font_scale=1.3)

predictions_logit = np.array([logit(xi) for xi in predictions[:,1]])
# Density distribution of propensity score (logic) broken down by treatment status
fig, ax = plt.subplots(1,2)
fig.suptitle('Density distribution plots for propensity score and logit(propensity score).')
sns.kdeplot(x = predictions[:,1], hue = T , ax = ax[0])
ax[0].set_title('Propensity Score')
sns.kdeplot(x = predictions_logit, hue = T , ax = ax[1])
ax[1].axvline(-0.85, ls='--')
ax[0].axvline(0.3, ls='--')
ax[1].set_title('Logit of Propensity Score')
plt.show()

```
The graph on the right (logit_propensity_score) demonstrates the density for each treatment status. There is overlap accross the range of values (-7,2.5). However on the left of "-0.85" there are a lot more 0's than 1's. On the right side of "-0.85", the opposite is true (a lot more 1's than 0's).

This will affect later how we will perform the matching so we can have balanced groups. In practise, this means that for values X > -0.85, there are less untreated samples than treated. This will lead to untreated samples being used for more than one treated.
```

In [ ]:
# Currently this does not affect the results as all observations fall within this range.
common_support = (predictions_logit > -10) & (predictions_logit < 10)

In [ ]:
# We generate a copy of X dataframe as out export CSV dataframe
# Since we exclude month_sales_clean_log / MeituanRun / MeituanFlash/ Merchant in the X above 
# We gonna add them back to the export CSV dataframe
PSMDF = X.copy()
PSMDF.loc[:,'propensity_score'] = predictions[:,1]
PSMDF.loc[:,'propensity_score_logit'] = predictions_logit
PSMDF.loc[:,'month_sales_clean_log'] = dfclean.month_sales_clean_log
PSMDF.loc[:,'month_sales_clean'] = dfclean.month_sales_clean
PSMDF.loc[:,'MeituanFlash'] = dfclean.MeituanFlash
PSMDF.loc[:,'MeituanRun'] = dfclean.MeituanRun
PSMDF.loc[:,'Merchant'] = dfclean.Merchant

In [ ]:
# Export the data
# PSMDF.to_csv('Merchant_NonMerchant_CatergoryAll.csv')

## 5-4 P-Score Matching
### Treatment = Merchant Vs MeituanFlash 
### Subgroup= Foods of  Category1
### Y = month_sale_clean

#### 5-4-1: P-Score

In [ ]:
# Define X & Y for P-score prediction

# Base on the LassoCV, we extract the feature that is non zero
# Remind that we will include all the dummy variable of category1 feature regardless the lasso result
Important_Feature = ['score',
                     'comment_number',
                     'avg_price_rmb',
                    'delivery_time_clean',
                    'in_time_delivery_percent',
                     'min_price_rmb',
                    'shipping_fee_clean',
                    'Day_Time',
                    'Night_Time',
                    'Mid_Night_Time',
                    'recommend_Bin',
                     'district population in (10 thousands)',
                     'population density (people/square kilometer)',
                     'Rent',
                     'Female',
                     '18 and below',
                     '19-24',
                     '25-34',
                     '35-44',
                     '45-54',
                     '55 and above',
                     '50-100 yuan',
                     '300 yuan',
                     'Shopping',
                     'Catering Service',
                     'Car Service',
                     'Leisure',
                     'Other',
                     'Medicare',
                    'MeituanFlash',
                    'MeituanRun',
                     'DessertDrink',
                     'DrugsDeliveried',
                    'Foods',
                     'FruitVegetables',
                    'MarketConvenience',
                     'RomanticFlowers',
                    'SweetCake',]

# For X
# 1: Regarding the input variable, we use important feature generated from above lasso regression where coefficient is non-zero
# 2: Make sure it MUST NOT contain the treatment
#    Therefore we drop 'MeituanRun' and 'MeituanFlash', cause we are predicting the delivery type as our P-Score(Which is same as Merchant)
# 3: Since the Output of Matching analysis is the monthly sales data, we need to make sure that X MUST NOT contain 'month_sales_clean' and 'month_sales_clean_log' 
# 4: Also, the subgroup is Foods, so we only incldue Foods + DessertDrink + SweetCake
# 5: After filter out non-food category dummy variables, remove them from the X dataframe
Important_Feature.remove('MeituanRun')
Important_Feature.remove('MeituanFlash')
X = dfclean[dfclean.MeituanRun==0][Important_Feature]
X = X[X['MarketConvenience']!=1][X['FruitVegetables']!=1][X['RomanticFlowers']!=1][X['DrugsDeliveried']!=1]
X.drop(['MarketConvenience','FruitVegetables','RomanticFlowers','DrugsDeliveried'],inplace=True,axis=1)


# For T
# The treatment we use is Merchant
# 1 = Merchant 0 = Non-Merchant(Which is MeituanFlash,since we exclude MeituanRun)
# Estimatae the Treatment Effect between Merchant and MeituanFlash (Exclude MeituanRun!!) 
T = dfclean[dfclean['MarketConvenience']!=1][dfclean['FruitVegetables']!=1][dfclean['RomanticFlowers']!=1][dfclean['DrugsDeliveried']!=1][dfclean['MeituanRun']!=1].Merchant

In [ ]:
# Design pipline 
#numeric_features  = ['score', 'comment_number', 'avg_price_rmb', 'delivery_time_clean',
#       'in_time_delivery_percent', 'min_price_rmb', 'shipping_fee_clean']
numeric_features= ['score',
                    'comment_number',
                    'avg_price_rmb',
                    'delivery_time_clean',
                    'in_time_delivery_percent',
                    'min_price_rmb',
                    'shipping_fee_clean',
                    'district population in (10 thousands)',
                    'population density (people/square kilometer)',
                    'Rent',
                    'Female',
                    '18 and below',
                    '19-24',
                    '25-34',
                    '35-44',
                    '45-54',
                    '55 and above',
                    '50-100 yuan',
                    '300 yuan',
                    'Shopping',
                    'Catering Service',
                    'Car Service',
                    'Leisure',
                    'Other',
                    'Medicare']


numeric_transformer  = Pipeline(steps=[('Scaler',StandardScaler())])
preprocessor         = ColumnTransformer(transformers=[('num',numeric_transformer,numeric_features)])
clf                  = Pipeline(steps=[('preprocessor',preprocessor),("classifier",lr())])
grid                 = {"classifier__C": np.arange(1,10,0.1).tolist()}
grid_search          = GridSearchCV(clf,grid,cv=10)
grid_search.fit(X,T)

In [ ]:
# Use the optimal parameter in logist
# Output the prediction base on the best parameter
BestPip = Pipeline(steps=[('preprocessor',preprocessor),
                          ("classifier",lr(C=grid_search.best_params_['classifier__C']))])
BestPip.fit(X, T)
predictions = BestPip.predict_proba(X)
predictions_binary = BestPip.predict(X)

# Eavluate the model
print('Accuracy: {:.4f}\n'.format(metrics.accuracy_score(T, predictions_binary)))
print('Precision: {:.4f}\n'.format(metrics.precision_score(T, predictions_binary)))
print('Confusion matrix:\n{}\n'.format(metrics.confusion_matrix(T, predictions_binary)))
print('F1 score is: {:.4f}'.format(metrics.f1_score(T, predictions_binary)))

In [ ]:
def logit(p):
    logit_value = math.log(p / (1-p))
    return logit_value
sns.set(rc={'figure.figsize':(16,10)}, font_scale=1.3)

predictions_logit = np.array([logit(xi) for xi in predictions[:,1]])
# Density distribution of propensity score (logic) broken down by treatment status
fig, ax = plt.subplots(1,2)
fig.suptitle('Density distribution plots for propensity score and logit(propensity score).')
sns.kdeplot(x = predictions[:,1], hue = T , ax = ax[0])
ax[0].set_title('Propensity Score')
sns.kdeplot(x = predictions_logit, hue = T , ax = ax[1])
ax[1].axvline(-0.5, ls='--')
ax[0].axvline(0.367, ls='--')
ax[1].set_title('Logit of Propensity Score')
plt.show()

In [ ]:
# We generate a copy of X dataframe as out export CSV dataframe
# Since we exclude month_sales_clean_log / MeituanRun / MeituanFlash/ Merchant in the X above 
# We gonna add them back to the export CSV dataframe
PSMDF = X.copy()
PSMDF.loc[:,'propensity_score'] = predictions[:,1]
PSMDF.loc[:,'propensity_score_logit'] = predictions_logit
PSMDF.loc[:,'month_sales_clean_log'] = dfclean[dfclean['MarketConvenience']!=1][dfclean['FruitVegetables']!=1][dfclean['RomanticFlowers']!=1][dfclean['DrugsDeliveried']!=1][dfclean['MeituanRun']!=1].month_sales_clean_log
PSMDF.loc[:,'month_sales_clean'] = dfclean[dfclean['MarketConvenience']!=1][dfclean['FruitVegetables']!=1][dfclean['RomanticFlowers']!=1][dfclean['DrugsDeliveried']!=1][dfclean['MeituanRun']!=1].month_sales_clean
PSMDF.loc[:,'MeituanFlash'] = dfclean[dfclean['MarketConvenience']!=1][dfclean['FruitVegetables']!=1][dfclean['RomanticFlowers']!=1][dfclean['DrugsDeliveried']!=1][dfclean['MeituanRun']!=1].MeituanFlash
PSMDF.loc[:,'MeituanRun'] = dfclean[dfclean['MarketConvenience']!=1][dfclean['FruitVegetables']!=1][dfclean['RomanticFlowers']!=1][dfclean['DrugsDeliveried']!=1][dfclean['MeituanRun']!=1].MeituanRun
PSMDF.loc[:,'Merchant'] = dfclean[dfclean['MarketConvenience']!=1][dfclean['FruitVegetables']!=1][dfclean['RomanticFlowers']!=1][dfclean['DrugsDeliveried']!=1][dfclean['MeituanRun']!=1].Merchant

In [ ]:
# Export the data
# PSMDF.to_csv('Merch_Flash_CatergoryFood.csv')

## 5-5 P-Score Matching
### Treatment = Merchant Vs MeituanFlash 
### Subgroup= Foods of  Category1
### Y = avg_price

#### 5-5-1: P-Score

In [ ]:
# Define X & Y for P-score prediction

# Base on the LassoCV, we extract the feature that is non zero
# Remind that we will include all the dummy variable of category1 feature regardless the lasso result
Important_Feature = ['score',
                     'comment_number',
                     'avg_price_rmb',
                    'delivery_time_clean',
                    'in_time_delivery_percent',
                     'min_price_rmb',
                    'shipping_fee_clean',
                    'Day_Time',
                    'Night_Time',
                    'Mid_Night_Time',
                    'recommend_Bin',
                     'district population in (10 thousands)',
                     'population density (people/square kilometer)',
                     'Rent',
                     'Female',
                     '18 and below',
                     '19-24',
                     '25-34',
                     '35-44',
                     '45-54',
                     '55 and above',
                     '50-100 yuan',
                     '300 yuan',
                     'Shopping',
                     'Catering Service',
                     'Car Service',
                     'Leisure',
                     'Other',
                     'Medicare',
                    'MeituanFlash',
                    'MeituanRun',
                     'DessertDrink',
                     'DrugsDeliveried',
                    'Foods',
                     'FruitVegetables',
                    'MarketConvenience',
                     'RomanticFlowers',
                    'SweetCake',]
# For X
# 1: Regarding the input variable, we use important feature generated from above lasso regression where coefficient is non-zero
# 2: Make sure it MUST NOT contain the treatment
#    Therefore we drop 'MeituanRun' and 'MeituanFlash', cause we are predicting the delivery type as our P-Score(Which is same as Merchant)
# 3: Since the Output of Matching analysis is the avg_price_rmb data, we need to make sure that X MUST NOT contain 'avg_price_rmb'
# 4: And also, different from above, month_sales_clean is no longer the Y(output) variable, so we will add month_sales_clean back to the feature list
# 5: Also, the subgroup is Foods, so we only incldue Foods + DessertDrink + SweetCake
# 6: After filter out non-food category dummy variables, remove them from the X dataframe
Important_Feature.remove('MeituanRun')
Important_Feature.remove('MeituanFlash')
Important_Feature.remove('avg_price_rmb')
Important_Feature.append('month_sales_clean')
X = dfclean[dfclean.MeituanRun==0][Important_Feature]
X = X[X['MarketConvenience']!=1][X['FruitVegetables']!=1][X['RomanticFlowers']!=1][X['DrugsDeliveried']!=1]
X.drop(['MarketConvenience','FruitVegetables','RomanticFlowers','DrugsDeliveried'],inplace=True,axis=1)


# For T
# The treatment we use is Merchant
# 1 = Merchant 0 = Non-Merchant(Which is MeituanFlash,since we exclude MeituanRun )
# Estimatae the Treatment Effect between Merchant and Non-Merchant (Exclude MeituanRun!!) 
T = dfclean[dfclean['MarketConvenience']!=1][dfclean['FruitVegetables']!=1][dfclean['RomanticFlowers']!=1][dfclean['DrugsDeliveried']!=1][dfclean['MeituanRun']!=1].Merchant

In [ ]:
# Design pipline 
# Replaced avg_price wtih month_sales_clean 
# numeric_features  = ['score', 'comment_number', 'month_sales_clean', 'delivery_time_clean',
#       'in_time_delivery_percent', 'min_price_rmb', 'shipping_fee_clean']

numeric_features= ['score',
                    'comment_number',
                    'month_sales_clean',
                    'delivery_time_clean',
                    'in_time_delivery_percent',
                    'min_price_rmb',
                    'shipping_fee_clean',
                    'district population in (10 thousands)',
                    'population density (people/square kilometer)',
                    'Rent',
                    'Female',
                    '18 and below',
                    '19-24',
                    '25-34',
                    '35-44',
                    '45-54',
                    '55 and above',
                    '50-100 yuan',
                    '300 yuan',
                    'Shopping',
                    'Catering Service',
                    'Car Service',
                    'Leisure',
                    'Other',
                    'Medicare']


numeric_transformer  = Pipeline(steps=[('Scaler',StandardScaler())])
preprocessor         = ColumnTransformer(transformers=[('num',numeric_transformer,numeric_features)])
clf                  = Pipeline(steps=[('preprocessor',preprocessor),("classifier",lr())])
grid                 = {"classifier__C": np.arange(1,10,0.1).tolist()}
grid_search          = GridSearchCV(clf,grid,cv=10)
grid_search.fit(X,T)

In [ ]:
# Use the optimal parameter in logist
# Output the prediction base on the best parameter
BestPip = Pipeline(steps=[('preprocessor',preprocessor),
                          ("classifier",lr(C=grid_search.best_params_['classifier__C']))])
BestPip.fit(X, T)
predictions = BestPip.predict_proba(X)
predictions_binary = BestPip.predict(X)

# Eavluate the model
print('Accuracy: {:.4f}\n'.format(metrics.accuracy_score(T, predictions_binary)))
print('Precision: {:.4f}\n'.format(metrics.precision_score(T, predictions_binary)))
print('Confusion matrix:\n{}\n'.format(metrics.confusion_matrix(T, predictions_binary)))
print('F1 score is: {:.4f}'.format(metrics.f1_score(T, predictions_binary)))

In [ ]:
def logit(p):
    logit_value = math.log(p / (1-p))
    return logit_value
sns.set(rc={'figure.figsize':(16,10)}, font_scale=1.3)

predictions_logit = np.array([logit(xi) for xi in predictions[:,1]])
# Density distribution of propensity score (logic) broken down by treatment status
fig, ax = plt.subplots(1,2)
fig.suptitle('Density distribution plots for propensity score and logit(propensity score).')
sns.kdeplot(x = predictions[:,1], hue = T , ax = ax[0])
ax[0].set_title('Propensity Score')
sns.kdeplot(x = predictions_logit, hue = T , ax = ax[1])
ax[1].axvline(-0.5, ls='--')
ax[0].axvline(0.367, ls='--')
ax[1].set_title('Logit of Propensity Score')
plt.show()

In [ ]:
# We generate a copy of X dataframe as out export CSV dataframe
# Since we exclude month_sales_clean_log / MeituanRun / MeituanFlash/ Merchant in the X above 
# We gonna add them back to the export CSV dataframe
PSMDF = X.copy()
PSMDF.loc[:,'propensity_score'] = predictions[:,1]
PSMDF.loc[:,'propensity_score_logit'] = predictions_logit
PSMDF.loc[:,'avg_price_rmb'] = dfclean[dfclean['MarketConvenience']!=1][dfclean['FruitVegetables']!=1][dfclean['RomanticFlowers']!=1][dfclean['DrugsDeliveried']!=1][dfclean['MeituanRun']!=1].avg_price_rmb
PSMDF.loc[:,'MeituanFlash'] = dfclean[dfclean['MarketConvenience']!=1][dfclean['FruitVegetables']!=1][dfclean['RomanticFlowers']!=1][dfclean['DrugsDeliveried']!=1][dfclean['MeituanRun']!=1].MeituanFlash
PSMDF.loc[:,'MeituanRun'] = dfclean[dfclean['MarketConvenience']!=1][dfclean['FruitVegetables']!=1][dfclean['RomanticFlowers']!=1][dfclean['DrugsDeliveried']!=1][dfclean['MeituanRun']!=1].MeituanRun
PSMDF.loc[:,'Merchant'] = dfclean[dfclean['MarketConvenience']!=1][dfclean['FruitVegetables']!=1][dfclean['RomanticFlowers']!=1][dfclean['DrugsDeliveried']!=1][dfclean['MeituanRun']!=1].Merchant

In [ ]:
# Export the data
# PSMDF.to_csv('Merch_Flash_CatergoryFood(Y=AvgPrice).csv')